<h3>RNN application: Generating names by implementing character level model</h3>

In [54]:
import numpy as np
import random
import pprint

In [55]:
data = open('dinos.txt', 'r').read()
data = data.lower()
chars = sorted(list(set(data)))
data_size, vocab_size = len(data), len(chars)

char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

<h4>Utility functions</h4>

In [56]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def smooth(loss, cur_loss):
    return loss * 0.999 + cur_loss * 0.001

def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    print ('%s' % (txt, ), end='')
    

def get_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    return txt

def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def initialize_parameters(n_a, n_x, n_y):
    np.random.seed(1)
    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    b = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    
    return parameters

def rnn_step_forward(parameters, a_prev, x):
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b) # hidden state
    p_t = softmax(np.dot(Wya, a_next) + by) # unnormalized log probabilities for next chars # probabilities for next chars 
    
    return a_next, p_t

def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients

def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    return parameters

def rnn_forward(X, Y, a0, parameters, vocab_size = 27):
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    loss = 0
    
    for t in range(len(X)):
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)
        
    return loss, cache

def rnn_backward(X, Y, parameters, cache):
    gradients = {}
    
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']

    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])

    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a

<h4>Gradient clipping</h4>

In [57]:
def clip(gradients, maxValue):
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
    for grad in [dWaa, dWax, dWya, db, dby]:
        np.clip(grad, -maxValue, maxValue, out=grad)
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    return gradients
    

<h4>Sampling</h4>
<h5>Picking the next word randomly among the most likely words</h5>

In [58]:
def sample(parameters, char_to_ix):
    Waa, Wax, Wya, b, by = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['b'], parameters['by']
    vocab_size = len(char_to_ix)
    n_a = Waa.shape[1]

    x = np.zeros((vocab_size, 1))
    a_prev = np.zeros((n_a, 1))

    indices = []
    idx = -1
    counter = 0
    newline_character = char_to_ix['\n']

    while (idx != newline_character and counter != 35):
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)

        idx = np.random.choice(list(range(vocab_size)), p = y.ravel())

        indices.append(idx)

        x = np.zeros((vocab_size, 1))
        x[idx] = 1

        a_prev = a
        counter += 1 

    if (counter == 35):
        indices.append(char_to_ix['\n'])

    return indices

<h4>Building the language model</h4>

In [59]:
def model(data, ix_to_char, char_to_ix, num_iterations=50000, n_a=50, dino_names=7, vocab_size=27, learning_rate=0.001):
    a_prev = np.zeros((n_a, 1))
    n_x, n_y = vocab_size, vocab_size

    parameters = initialize_parameters(n_a, n_x, n_y)
    loss = get_initial_loss(vocab_size, dino_names)

    examples = data.split('\n')
    examples = [x.lower().strip() for x in examples]

    np.random.seed(0)
    np.random.shuffle(examples)

    for j in range(num_iterations):
        index = j%len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]]
        Y = X[1:] + [char_to_ix["\n"]]

        curr_loss, cache = rnn_forward(X, Y, a_prev, parameters)
        gradients, a = rnn_backward(X, Y, parameters, cache)
        gradients = clip(gradients, 5)
        parameters = update_parameters(parameters, gradients, learning_rate)

        loss = smooth(loss, curr_loss)

        if j%2000 == 0:
            print('Iteration: %d, Loss: %f' %(j, loss) + '\n')
            for name in range(dino_names):
                sampled_indices = sample(parameters, char_to_ix)
                print_sample(sampled_indices, ix_to_char)

            print('\n')

    return parameters


In [60]:
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.087336

Co
Eyhqgyshubjerbjgmis
Qfguyhvbqtdkxb
Okuxhjmozrjgycsxqkeisqywvxgijorg
Jhywwqinvbuavuapmddtozzzcyflteoebci

Kqjbagdzduoomziunjypckurxgtloyzwiwq




Iteration: 2000, Loss: 36.108329

L
Sosssftonr
Afyah
Hhvuk
Urazaufmsrurauss
Katsuxszr



Iteration: 4000, Loss: 34.975822

Htuw
Aukoyjsdtpcdcrsustannsailus
Aisossarrtunusmtu
As
Nder
Uonorgaiarhuusouus
Idnzuc


Iteration: 6000, Loss: 30.887714

Uraroeeafahr
Acfgerzothaneprrusaunos
Aurosospurus
Wgytjsosatruausltis
Psalaususaurus
Uyafaahltcotosaus
Diosausausus


Iteration: 8000, Loss: 28.841787

Ruhintaxs
Lortreos
Cigrnisoys
Ataktusairur
Eiinoneaais
Rpsiyses
Zarpidasaus


Iteration: 10000, Loss: 27.837348

Vheuydoepscurusaul
Aeilosoyrus
Ta
Elnaosaurusoumup
Ccjonunosaurus
Ipaesomhusaunus
Taggcosaurur


Iteration: 12000, Loss: 26.914573

Qanirusianir
Gvelasaurus
Spbdoleeeohas
Ianrodosausup
Ceononanturus
Cnaxosaurus
Calecrostes


Iteration: 14000, Loss: 26.542034

Harashasaurus
Meweannapsausus
Lanysanrus
Nanglgsonqos
Rxagosaurus
Eluhirunul
Kcemstosaurus


Iteration: 16000, Loss: 26.063137

Blcpecaui
Terrius
Norarosaurus
Tocigeleron
Andauratxi
Alsauasaurus
Ofaanusaurus


Iteration: 18000, Los